In [ ]:
%matplotlib qt

In [ ]:
import os.path as op
import mne
from mne.preprocessing import ICA
from mne.preprocessing import annotate_movement, compute_average_dev_head_t,annotate_muscle_zscore
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np

import matplotlib

In [ ]:
sub_ids = [
            'pre4013_y'
           ]
data_path = '.\data\\1raw_rename\\'

'''

https://mne.discourse.group/t/where-i-can-find-a-montage-for-neuroscan-64ch/4179/3

'''
sub_id = sub_ids[0]
fname = op.join(data_path + sub_id + '.fif')
raw = mne.io.read_raw_fif(fname, preload=True, verbose=None)
# raw.plot_sensors()
# raw.plot(n_channels = 50, duration =40, scalings=60e-6, remove_dc = True,
#                  highpass=0.1, lowpass=30)
# raw.plot(n_channels = 50, duration =40, scalings=60e-6, remove_dc = True,)
# raw.plot_sensors()
# raw.plot()
# 这两个channel其实是有用的,但是和模板里名字不一致.
raw.drop_channels(['CB1','CB2', 'Trigger','EKG', 'EMG']) # i dont need those channels
rename_dict = {'HEO': 'HEOG','VEO':'VEOG','FP1':'Fp1', 'FPZ':'Fpz', 'FP2':'Fp2', 'FZ':'Fz', 'FCZ':'FCz', 'CZ':'Cz', 'CPZ':"CPz", 'PZ':"Pz", 'POZ':"POz", 'OZ':"Oz"}
raw.rename_channels(rename_dict)

print(raw.info)

chan_types_dict = { "VEOG":"eog","HEOG":"eog"}
raw.set_channel_types(chan_types_dict)

#电极点位置确定
raw.plot_sensors() # 画出电极位置
locs_info_path = './standard-10-5-cap385.elp'
# montage = mne.channels.read_custom_montage(locs_info_path, head_size=0.095)
montage = mne.channels.make_standard_montage(kind='standard_1005')
# 自带的montage,如果没有montage文件. 如果没有就用mne自带的
# mne.channels.get_builtin_montages()
raw.set_montage(montage)
raw.plot_sensors()# 对比一下 montage之后的位置
plt.show() # 只呈现一次


In [ ]:
'''
在进行filter之前,查找muscle,可以先把工频给删除掉.
之所以要在filter之前,是因为muscle算法的判断就是高频的震荡(比如移动身体导致的皮肤和电极的摩擦)
'''
raw_muscle = raw.copy()
raw_muscle.notch_filter([50, 100, 150, 200])
threshold_muscle = 3  # z-score  设置得稍微小一点,选出更多的范围.不过这样子可能导致bad区间里面也有眼电成分.如果有一半标红了,那就没意义了..autoreject对眼电没辙.
# Choose one channel type, if there are axial gradiometers and magnetometers,
# select magnetometers as they are more sensitive to muscle activity.
# min_length_good=0.1 两个bad之间如果很短,就之间标为bad范围.
# filter_freq=[100, 150] 频率再设置宽一点.
annot_muscle, scores_muscle = annotate_muscle_zscore(
    raw_muscle, ch_type="eeg", threshold=threshold_muscle, min_length_good=1,
    filter_freq=[100, 150],n_jobs=4) # 选择区间是110-140HZ,这个已经远超神经信号的频率了
# muscle的marker加上原来的marker
raw_muscle.set_annotations(annot_muscle+raw.annotations)
raw_muscle.plot(n_channels = 50, duration =40, scalings=60e-6, remove_dc = True,
                 highpass=0.1, lowpass=30)
'''
查看 muscle score 的分布情况 在时间线上.
'''
fig, ax = plt.subplots()
ax.plot(raw_muscle.times, scores_muscle)
ax.axhline(y=threshold_muscle, color='r')
ax.set(xlabel='time, (s)', ylabel='zscore', title='Muscle activity')

In [ ]:

'''
自动化截取RSA部分的数据

找到RSA实验的最前和最后的marker,截止时间

截取RSA的RAW数据
'''
# 读取和打印marker
# events, event_id = mne.events_from_annotations(raw)
# for marker_name in event_id:
#     print(marker_name)

# 确定RSA的marker
list_RSAmarker = ['101', '102', '103', '104', '111', '112', '113', '114', '121', '122', '123', '124', '131', '132', '133', '134', ]

# 提取原始数据中的所有事件
events, event_id = mne.events_from_annotations(raw_muscle)

# 通过遍历list_RSAmarker列表，获取所需事件的索引
selected_event_indices = [event_id[str(marker)] for marker in list_RSAmarker]

# 提取与list_RSAmarker中的标记对应的事件
selected_events = np.array([event for event in events if event[2] in selected_event_indices])

# 找到最早和最晚出现的onset time

first_onset_time = raw_muscle.times[selected_events[0, 0]]-2 #第一个marker的时间再往前2s
# 给出2的trail空间
last_onset_time = raw_muscle.times[selected_events[-1, 0]]+2 #最后一个marker的时间再往后2s

print(f"First onset time: {first_onset_time} s")
print(f"Last onset time: {last_onset_time} s")

raw_crop  = raw_muscle.copy().crop(tmin=first_onset_time, tmax=last_onset_time)


In [ ]:
#找出坏导
%matplotlib qt
# raw.plot(n_channels = 50, duration =40, scalings=60e-6, remove_dc = True,
#                  highpass=0.1, lowpass=30)
raw_crop.plot(n_channels = 50, duration =40, scalings=60e-6, remove_dc = True,)


In [ ]:
print(raw_crop.info['bads'])
# save_path_bad = '.\data\\2raw_bad\\'
# raw_crop.save(save_path_bad + sub_id + '.fif', overwrite=True)

In [ ]:
#插值
original_bads = deepcopy(raw_crop.info['bads'])
print(raw_crop.info['bads'])
#截取一部分作图
# raw.copy().crop(tmin=0,tmax=100).plot(n_channels = 50, duration =40, scalings=40e-6,remove_dc = True,highpass=1,lowpass=40)
# How interpolation works¶根据手册，是根据topo位置来插值？所以需要location
raw_interp = raw_crop.copy().interpolate_bads(reset_bads=True)
print(raw_interp.info['bads'])
# raw_interp.copy().crop(tmin=0,tmax=100).plot(n_channels = 50, duration =40, scalings=40e-6, remove_dc = True,highpass=1,lowpass=40)

#滤波
# raw_interp.copy().plot_psd(area_mode='range', average=False)
# 要做时频分析，上限高一点？脑干 80
# 不同数据对这个要求不一样,这里稍微设置高一点,后面再各自处理好了.
raw_middlepass = raw_interp.copy().filter(0.1, 40.)
sample_rate=500
freqs = np.arange(50, sample_rate / 2, 50)
raw_middlepass_notch = raw_middlepass.copy().notch_filter(freqs=freqs)
# raw_middlepass_notch.copy().plot_psd(area_mode='range', average=False)

#重参考
raw_rerefer = raw_middlepass_notch.copy().set_eeg_reference(ref_channels='average')
#保存,在ICA之前
raw_rerefer.plot(n_channels = 50, duration =40, scalings=60e-6, remove_dc = True,)

'''
标注出bad区间,提高ICA的眼电成分更加集中
'''

break_annots = mne.preprocessing.annotate_break(raw_rerefer,min_break_duration=20,)
# 防止错位
# 必须放在annotation_break之后
raw_rerefer.annotations.onset = raw_rerefer.annotations.onset - first_onset_time
raw_rerefer.set_annotations(annotations=raw_rerefer.annotations + break_annots)
# raw_rerefer.plot(n_channels = 50, duration =40, scalings=60e-6)


In [ ]:
# 根据amplitude标注一次 然后手动去调整宽度.
'''
注意 amplitude标注并不是标注出所有的bad区间,其原理是根据 幅值增长速度 而非 幅度本身

因此其作用应该是标注出幅值快速变化的时间点,你要去画出来之后,在EEG的图上,留下这些bad marker,自己人为地去扩宽一下.

'''
# peak的选择可以严格一点.理论上来说眼电的幅值也就20uV
amp_annots = mne.preprocessing.annotate_amplitude(raw_rerefer, peak=5e-6, min_duration=0.05)[0]
# 在原有基础上加上amp_annots
raw_rerefer.annotations.onset = raw_rerefer.annotations.onset - first_onset_time
raw_rerefer.set_annotations(raw_rerefer.annotations+amp_annots)
raw_rerefer.plot(n_channels = 50, duration =40, scalings=60e-6)


In [ ]:
'''
问题，不知道ICA到底有没有吧 break计算上。
'''
import time
time_start=time.time()
# ICA
raw_filt = raw_rerefer.copy()
raw_filt.load_data().filter(l_freq=0.1, h_freq= 40)
# 有一段数据非常糟糕,而且是有mark的部分,所以直接截掉,而不是用break来标记
# raw_filt.crop(tmin=132)
raw_filt.plot(n_channels = 50, duration =40, scalings=60e-6)

In [ ]:
ica = ICA(n_components=50, random_state=97, )
#训练，但是并没有删除
ica.fit(raw_filt)
#计时
time_end=time.time()
print('time cost',time_end-time_start,'s')
print(sub_id)

In [ ]:
#有多种选择，自动，手动。找出eog component
# eog_inds, eog_scores = ica.find_bads_eog(raw_filt, threshold=3.0, ch_name='Fp2')#0 7数据眼电有问题
eog_inds, eog_scores = ica.find_bads_eog(raw_filt, threshold='auto') # 用3是因为并不是真的那么准确,你需要人为判断.如果改成2,,可能会自动排除许多component.

#参考一下机器
ica.plot_scores(eog_scores)
#但是要自己检查一下,有问题就手动修改eog_ins

In [ ]:
ica.plot_components(inst=raw_filt)  #一跑就卡死

In [ ]:
'''
上面是找眼电成分,这里再找一次muscle成分.

你会发现muscle渗透到了很多成分当中.甚至超过一半,但是你这部分的脚本仅仅是挑出来眼电.
即把眼电逼出来,尽量避免混合 眼电+muscle


'''
muscle_idx_auto, scores = ica.find_bads_muscle(raw)
ica.plot_scores(scores, exclude=muscle_idx_auto)
print(f'Automatically found muscle artifact ICA components: {muscle_idx_auto}')

In [ ]:

# 手动
eog_inds = [1,3]

In [ ]:
#eog_inds = [0,1]
ica.exclude = eog_inds
ica.plot_scores(eog_scores)
#这个……在filt_raw训练出来的model，可以直接给别的数据用。。
reconst_raw =raw_filt.copy()
reconst_raw.plot(n_channels = 50, duration =40, scalings=40e-6)
ica.apply(reconst_raw)
reconst_raw.plot(n_channels = 50, duration =40, scalings=40e-6)

In [ ]:
#确认无误，保存一下,raw ica
save_path_ica = '.\data\\4raw_ica\\'
ica.save(save_path_ica + sub_id + '.fiff', overwrite=True)
reconst_raw.save(save_path_ica + sub_id + '.fif', overwrite=True)

In [ ]:
'''
脑区域图。

'''
raw.plot_sensors(show_names=True)